# Driver Overview

Let's run the WPS prep utility `ungrib` to transform a GFS 0-hour forecast GRIB files into initial conditions suitable for WRF or MPAS.

First `cd` into the `configs/` directory:

In [1]:
import os, shutil
os.chdir("../configs")
shutil.rmtree("driver-overview", ignore_errors=True)

Here's the config we'll use. Note the additional `platform:` and `ungrib.execution.batchargs:` blocks, which will be necessary for running on the batch system:

In [2]:
!cat do00.yaml

data:
  '{{ val.gfs }}': https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.{{ val.yyyymmdd }}/{{ val.hh }}/atmos/gfs.t{{ val.hh }}z.pgrb2.0p25.f000
platform:
  account: wrfruc
  scheduler: slurm
ungrib:
  execution:
    batchargs:
      cores: 1
      walltime: "00:01:00"
    executable: '{{ "CONDA_PREFIX" | env }}/bin/ungrib'
  gribfiles:
    - '{{ val.gfs }}'
  rundir: '{{ val.rundir }}'
  start: '{{ cycle }}'
  step: 1
  stop: '{{ cycle }}'
  vtable: '{{ "WPSFILES" | env }}/Vtable.GFS'
val:
  gfs: '{{ val.rundir }}/anl.grib2'
  hh: '{{ cycle.strftime("%H") }}'
  rundir: '{{ "PWD" | env }}/driver-overview'
  yyyymmdd: '{{ cycle.strftime("%Y%m%d") }}'


First, use `uw fs` to stage the GFS GRIB file from AWS:

In [3]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

[2025-09-22T20:45:30]     INFO Validating config against internal schema: files-to-stage
[2025-09-22T20:45:30]     INFO Schema validation succeeded for fs config
[2025-09-22T20:45:30]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Executing
[2025-09-22T20:45:45]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Ready
[2025-09-22T20:45:45]     INFO File copies: Ready


Now run `ungrib` via the UW driver, locally:

In [4]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T20:46:48]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:46:48]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:46:48]     INFO Validating config against internal schema: platform
[2025-09-22T20:46:48]     INFO Schema validation succeeded for platform config
[2025-09-22T20:46:48]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Executing
[2025-09-22T20:46:48]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Ready
[2025-09-22T20:46:48]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Executing
[2025-09-22T20:46:48]     INFO Schema validation succeeded for user-updated config
[2025-09-22T20:46:48]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Ready
[2025-09-22T2

See the output:

In [5]:
!tree driver-overview

driver-overview
├── anl.grib2
├── FILE:2025-09-21_06
├── GRIBFILE.AAA -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2
├── namelist.wps
├── runscript.ungrib
├── runscript.ungrib.done
├── runscript.ungrib.out
├── ungrib.log
└── Vtable -> /scratch4/BMC/rtruc/Paul.Madden/conda/envs/uwtools/etc/wps/Vtable.GFS

1 directory, 9 files


Note that an invocation again is a no-op:

In [6]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T20:47:25]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:47:25]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:47:25]     INFO Validating config against internal schema: platform
[2025-09-22T20:47:25]     INFO Schema validation succeeded for platform config
[2025-09-22T20:47:25]     INFO 20250921 06Z ungrib run: Ready


The `ungrib` driver views its `runscript.ungrib.done` file as evidence of its readiness, so we'll need to remove it to force re-execution. We'll also remove `runscript.ungrib` so that it gets recreated with batch directives -- remember, drivers are idempotent, and `runscript.ungrib` already exists. For brevity, we'll just remove `runscript.ungrib*`.

In [7]:
%%bash
rm -fv driver-overview/runscript.ungrib*
uw ungrib run -c do00.yaml --cycle 2025-09-21T06 --batch
squeue -u $USER

removed 'driver-overview/runscript.ungrib'
removed 'driver-overview/runscript.ungrib.done'
removed 'driver-overview/runscript.ungrib.out'


[2025-09-22T20:48:15]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:48:15]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:48:15]     INFO Validating config against internal schema: platform
[2025-09-22T20:48:15]     INFO Schema validation succeeded for platform config
[2025-09-22T20:48:15]     INFO 20250921 06Z ungrib runscript.ungrib: Executing
[2025-09-22T20:48:15]     INFO 20250921 06Z ungrib runscript.ungrib: Ready
[2025-09-22T20:48:15]     INFO 20250921 06Z ungrib provisioned run directory: Ready
[2025-09-22T20:48:15]     INFO 20250921 06Z ungrib run via batch submission: Executing
[2025-09-22T20:48:15]     INFO Running: sbatch /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/runscript.ungrib 2>&1 | tee /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/runscript.ungrib.submit in /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview
[2025-09-22T20:48:15]     INFO 202

     JOBID PARTITION  NAME                     USER             STATE        TIME TIME_LIMIT NODES NODELIST(REASON)
  16397116 hera       runscript.ungrib         Paul.Madden      PENDING      0:00       1:00     1 (None)


A little later...

In [8]:
%%bash
squeue -u $USER
tree driver-overview

     JOBID PARTITION  NAME                     USER             STATE        TIME TIME_LIMIT NODES NODELIST(REASON)
driver-overview
├── anl.grib2
├── FILE:2025-09-21_06
├── GRIBFILE.AAA -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2
├── namelist.wps
├── runscript.ungrib
├── runscript.ungrib.done
├── runscript.ungrib.out
├── runscript.ungrib.submit
├── ungrib.log
└── Vtable -> /scratch4/BMC/rtruc/Paul.Madden/conda/envs/uwtools/etc/wps/Vtable.GFS

1 directory, 10 files


Note the new batch directives in `runscript.ungrib`:

In [9]:
!cat driver-overview/runscript.ungrib

#!/bin/bash

#SBATCH --account=wrfruc
#SBATCH --chdir=/scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview
#SBATCH --ntasks=1
#SBATCH --output=runscript.ungrib.out
#SBATCH --time=00:01:00

time /scratch4/BMC/rtruc/Paul.Madden/conda/envs/uwtools/bin/ungrib
test $? -eq 0 && touch runscript.ungrib.done


We can also execute just parts of the driver's task graph. Let's see the available tasks:

In [10]:
!uw ungrib --help

usage: uw ungrib [-h] [--version] [--show-schema] TASK ...

Execute ungrib tasks

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --show-schema
      Show driver schema and exit

Positional arguments:
  TASK
    gribfiles
      Symlinks to all the GRIB files
    namelist_file
      The namelist file
    provisioned_rundir
      Run directory provisioned with all required content
    run
      A run
    runscript
      The runscript
    show_output
      Show the output to be created by this component
    validate
      Validate the UW driver config
    vtable
      A symlink to the Vtable file


Let's remove the run directory and this time ask just for the namelist file:

In [11]:
%%bash
rm -rf driver-overview
uw ungrib namelist_file -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T20:50:54]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:50:54]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:50:54]     INFO Validating config against internal schema: platform
[2025-09-22T20:50:54]     INFO Schema validation succeeded for platform config
[2025-09-22T20:50:54]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Executing
[2025-09-22T20:50:54]     INFO Schema validation succeeded for user-updated config
[2025-09-22T20:50:54]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/namelist.wps: Ready


In [12]:
!tree driver-overview

driver-overview
└── namelist.wps

1 directory, 1 file


In [13]:
cat driver-overview/namelist.wps

&share
    end_date = '2025-09-21_06:00:00'
    interval_seconds = 3600
    max_dom = 1
    start_date = '2025-09-21_06:00:00'
    wrf_core = 'ARW'
/

&ungrib
    out_format = 'WPS'
    prefix = 'FILE'
/


Expand that request to the full provisioned run directory:

In [14]:
!uw ungrib provisioned_rundir -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T20:51:26]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:51:26]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:51:26]     INFO Validating config against internal schema: platform
[2025-09-22T20:51:26]     INFO Schema validation succeeded for platform config
[2025-09-22T20:51:26]  WARNING File /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Not ready [external asset]
[2025-09-22T20:51:26]     INFO 20250921 06Z ungrib runscript.ungrib: Executing
[2025-09-22T20:51:26]     INFO 20250921 06Z ungrib runscript.ungrib: Ready
[2025-09-22T20:51:26]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/Vtable: Executing
[2025-09-22T20:51:26]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/Vtable: Ready
[2025-09-22T20:51:26]  WARNING 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training

Note two interesting things:
1. The namelist file was not recreated, because it was created by the previous invocation.
2. Since we removed the full run directory, including the GFS GRIB file, `uw` reports that the `provisioned_rundir` task is not complete: It needs the GRIB file to be complete. We can stage that, then re-run the driver:

In [15]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

[2025-09-22T20:51:40]     INFO Validating config against internal schema: files-to-stage
[2025-09-22T20:51:40]     INFO Schema validation succeeded for fs config
[2025-09-22T20:51:40]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Executing
[2025-09-22T20:52:25]     INFO HTTP https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20250921/06/atmos/gfs.t06z.pgrb2.0p25.f000 -> /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/anl.grib2: Ready
[2025-09-22T20:52:25]     INFO File copies: Ready


In [16]:
!uw ungrib provisioned_rundir -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T20:55:18]     INFO Validating config against internal schema: ungrib
[2025-09-22T20:55:18]     INFO Schema validation succeeded for ungrib config
[2025-09-22T20:55:18]     INFO Validating config against internal schema: platform
[2025-09-22T20:55:18]     INFO Schema validation succeeded for platform config
[2025-09-22T20:55:18]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Executing
[2025-09-22T20:55:18]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Ready
[2025-09-22T20:55:18]     INFO 20250921 06Z ungrib GRIB files: Ready
[2025-09-22T20:55:18]     INFO 20250921 06Z ungrib provisioned run directory: Ready


Now the run directory is fully provisioned. We could run ungrib with `uw ungrib run`, or even run its `runscript.ungrib` directly. For experimental purposes, we could even manually edit any of the run-directory files first, then execute the run.

In [17]:
!rm -rf driver-overview